In [83]:
import re
import json
import pickle

from sklearn.metrics import classification_report

In [ ]:
with open("/Utilisateurs/umushtaq/emotion_analysis_comics/comics_FT/saved_models/comics35_pg_llama-3-8b-Instruct-bnb-4bit/comics35_results_3.pickle", 'rb') as f:
    
    results = pickle.load(f)

In [ ]:
grounds = results['grounds']
predictions = results['predictions']

### Process Grounds

In [84]:
grounds = ['''### Output:
{"argument_types": ['M', 'M', 'C', 'C', 'C', 'P', 'P', 'P', 'P', 'P', 'P']}
{"related_arguments": [(8, 4), (7, 4), (6, 4), (10, 11), (9, 5), (11, 5)]}
{"relation_types": ['S', 'S', 'S', 'S', 'S', 'S']}
''']

In [101]:
def process_grounds(grounds):
    
    arg_types = []
    rel_pairs = []
    rel_types = []
    
    for ground in grounds:
        
        args = ground.split("\n")[1].replace("'", '"')
        arg_types.append(json.loads(args)["argument_types"])
        
        rels = re.sub(r'\((\d+), (\d+)\)', r'[\1, \2]', ground.split("\n")[2])
        related_args = [tuple(pair) for pair in json.loads(rels)["related_arguments"]]
        rel_pairs.append(related_args)
        
        
        rel_t = ground.split("\n")[3].replace("'", '"')
        rel_types.append(json.loads(rel_t)["relation_types"])
        
    return arg_types, rel_pairs, rel_types

In [ ]:
grounds_acc, grounds_ari, grounds_arc  = process_grounds(grounds)

### Process Predictions

In [ ]:
def process_predictions(predictions):
    
    arg_types = []
    rel_pairs = []
    rel_types = []
    
    for pred in predictions:
        
        args = pred.split("\n")[1].replace("'", '"')
        arg_types.append(json.loads(args)["argument_types"])
        
        rels = re.sub(r'\((\d+), (\d+)\)', r'[\1, \2]', pred.split("\n")[2])
        related_args = [tuple(pair) for pair in json.loads(rels)["related_arguments"]]
        rel_pairs.append(related_args)
        
        
        rel_t = pred.split("\n")[3].replace("'", '"')
        rel_types.append(json.loads(rel_t)["relation_types"])
        
    return arg_types, rel_pairs, rel_types

In [ ]:
predictions_acc, predictions_ari, predictions_arc  = process_predictions(predictions)

### Compute ACC metrics

In [ ]:
len(grounds_acc), len(predictions_acc)

In [ ]:
grounds = [elem for sublist in grounds_acc for elem in sublist]
predictions = [elem for sublist in predictions_acc for elem in sublist]

In [ ]:
print(classification_report(grounds, predictions, digits=3))

### Compute ARI metrics